## Identify the class for each item, using the hint

In [229]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU
from keras.optimizers import SGD

In [2]:
seqdataIn = pd.read_json('/Volumes/GoogleDrive/My Drive/NYU/PhD/20 DL/project/data/seq-public.json', orient='records')
print(seqdataIn)

                  stem               options            category    id
0           7,9,-1,5,?        [4, 2, -1, -3]            sequence     0
1        3,2,5/3,3/2,?  [1/4, 7/5, 3/4, 2/5]            sequence     1
2           1,2,5,26,?    [34, 841, 677, 37]            sequence     2
3            2,12,30,?      [50, 65, 75, 56]            sequence     3
4        2,1,2/3,1/2,?  [3/4, 1/4, 2/5, 5/6]            sequence     4
...                ...                   ...                 ...   ...
1071  20 22 25 30 37 ?                    []  sequence-reasoning  1090
1072        0 1 3 10 ?                    []  sequence-reasoning  1091
1073       5 15 10 215                    []  sequence-reasoning  1092
1074        1 2 5 29 ?    [34, 841, 866, 37]  sequence-reasoning  1093
1075         2 12 30 ?      [50, 65, 75, 56]  sequence-reasoning  1094

[1076 rows x 4 columns]


In [3]:
seqDataAns =pd.read_json('/Volumes/GoogleDrive/My Drive/NYU/PhD/20 DL/project/data/seq-public.answer.json',orient='index')
print(seqDataAns)

     answer                                               hint
0       [4]      A<sub>n+1</sub>=2<sup>5-n</sup>-A<sub>n</sub>
1       [2]  3/1, 4/2, 5/3, 6/4\n\nA<sub>n+1</sub>=A<sub>n<...
2       [3]        A<sub>n+1</sub>=A<sub>n</sub><sup>2</sup>+1
3       [4]                           A<sub>n</sub>=2n\*(2n-1)
4       [3]             4/2, 4/4, 4/6, 4/8\n\nA<sub>n</sub>=2n
...     ...                                                ...
1090     48                A<sub>n+1</sub>-A<sub>n</sub>=P(n);
1091    102           A<sub>n\*2</sub>=A<sub>n\*2-1</sub>^2+2;
1092   -115   A<sub>n</sub>=A<sub>n-2</sub>^2-A<sub>n-1</sub>;
1093    [3]  A<sub>n</sub>=A<sub>n-1</sub>^2+A<sub>n-2</sub...
1094    [4]                      A<sub>n</sub>=(2\*n-1)\*2\*n;

[1076 rows x 2 columns]


In [4]:
def hint2txt(h):
    return h.replace('<sub>','_{').replace('</sub>','}').replace('<sup>','^{').replace('</sup>','}').replace('\n',' | ').replace('\*','*')
seqDataAns['hint'] = seqDataAns['hint'].map(lambda x: hint2txt(x))
print(seqDataAns['hint'])

0                                   A_{n+1}=2^{5-n}-A_{n}
1       3/1, 4/2, 5/3, 6/4 |  | A_{n+1}=A_{n}+1 |  | B...
2                                     A_{n+1}=A_{n}^{2}+1
3                                         A_{n}=2n*(2n-1)
4                        4/2, 4/4, 4/6, 4/8 |  | A_{n}=2n
                              ...                        
1090                                  A_{n+1}-A_{n}=P(n);
1091                               A_{n*2}=A_{n*2-1}^2+2;
1092                             A_{n}=A_{n-2}^2-A_{n-1};
1093                           A_{n}=A_{n-1}^2+A_{n-2}^2;
1094                                   A_{n}=(2*n-1)*2*n;
Name: hint, Length: 1076, dtype: object


In [5]:
seqDataAns['hint1'] = [item.split('=') for item in seqDataAns['hint']] #split the hint by '=', to search item
                                                                        #in second term (after =)

In [6]:
seqDataAns['classes'] = 0

In [19]:
for i in range(len(seqDataAns)):
    if i in [82,  257, 298, 306, 451, 452, 463, 464,468, 476,495, 498,499, 500,504,505,623,624,626,627,628]:
        continue
        
    if seqDataAns['classes'][i] == 0:
        y = False
        z = False
        if len(seqDataAns['hint1'][i])>1:
            y = re.search('A_{', seqDataAns['hint1'][i][1])
            z = re.search('(?<!_).*?n', seqDataAns['hint1'][i][1])

        if (y) and not(z):
            seqDataAns.loc[i, 'classes'] = 1   #recursive A_{n}
            
        elif not(y) and (z):
            seqDataAns.loc[i, 'classes'] = 2   #just the position n
        
        elif not(y) and not(z):
            seqDataAns.loc[i, 'classes'] = 0   #none, or errors
            
        else:
            seqDataAns.loc[i, 'classes'] = 3   #both
            
        

In [20]:
seqDataAns[seqDataAns['classes'] == 0];

In [21]:
seqDataAns.classes.value_counts();

## Sequence parsing

In [22]:
sequences = seqdataIn['stem']
print(sequences)

0             7,9,-1,5,?
1          3,2,5/3,3/2,?
2             1,2,5,26,?
3              2,12,30,?
4          2,1,2/3,1/2,?
              ...       
1071    20 22 25 30 37 ?
1072          0 1 3 10 ?
1073         5 15 10 215
1074          1 2 5 29 ?
1075           2 12 30 ?
Name: stem, Length: 1076, dtype: object


In [23]:
#cleans up sequences
def txt2seq(seq):
    s2 = []
    for s in seq:
        si = re.sub("\(\s?\)","?",s)       #replace parenthesis with question mark
        a = re.split(',|\s|-',si)      #remove any delimiters
        a = [e for e in a if e != ""]  #remove empty strings
        s2.append(a)
    return s2
split_seq = txt2seq(sequences)
unsplit = np.array(split_seq)[list(map(lambda x: len(x) == 1, split_seq))]
print("Unsplit: " + str(unsplit))
print("Split: " + str(split_seq))

Unsplit: []
Split: [['7', '9', '1', '5', '?'], ['3', '2', '5/3', '3/2', '?'], ['1', '2', '5', '26', '?'], ['2', '12', '30', '?'], ['2', '1', '2/3', '1/2', '?'], ['4', '2', '2', '3', '6', '?'], ['1', '7', '8', '57', '?'], ['4', '12', '8', '10', '?'], ['1/2', '1', '1', '?', '9/11', '11/13'], ['95', '88', '71', '61', '50', '?'], ['2', '6', '13', '39', '15', '45', '23', '?'], ['1', '3', '3', '5', '7', '9', '13', '15', '?', '?'], ['1', '2', '8', '28', '?'], ['0', '4', '18', '?', '100'], ['1', '1', '2', '2', '3', '4', '3', '5', '?'], ['1', '52', '313', '174', '?'], ['5', '15', '10', '215', '?'], ['7', '0', '1', '2', '9', '?'], ['0', '1', '3', '10', '?'], ['9/2', '14', '65/2', '?', '217/2'], ['1', '1', '2', '6', '24', '?'], ['3', '4', '8', '24', '88', '?'], ['20', '22', '25', '30', '37', '?'], ['1/9', '2/27', '1/27'], ['√2', '3', '√28', '√65', '?'], ['1', '2', '4', '8', '16', '?'], ['2', '1', '2/3', '1/2', '?'], ['1', '1', '3', '7', '17', '41', '?'], ['5/2', '5', '25/2', '75/2', '?'], ['6', '

In [24]:
#checks for valid sequences (all numbers or fractions excluding ?)
def findBadSeq(seq):
    badSeqs = {}
    for a in range(len(seq)):     #check each sequence
        s = seq[a]
        for i in s:   #check each item (assuming sequence is already split)
            if(re.match(r'[^0-9\/\?]',i)):    #check if any words or weird characters in the sequence
                badSeqs[a] = s
                break
    return badSeqs

#remove bad sequences from good sequences
def remBadSeq(seq):
    badSeqInd = list(findBadSeq(seq).keys())
    goodSeq = {}
    for i in range(len(seq)):
        if i not in badSeqInd:     #want to keep the indexes to get the answer for it
            goodSeq[i] = seq[i]
    return goodSeq

In [25]:
# show bad sequences removed
bs = findBadSeq(split_seq)
for k,v in bs.items():
    print(str(k) + ":" + str(v))
print("# Bad sequences = " + str(len(bs)) + "/" + str(len(sequences)))

24:['√2', '3', '√28', '√65', '?']
85:['A', 'F', 'H', 'K', 'N', '?', '?']
86:['A', 'D', 'I', 'P', 'Y', 'CF', 'DI', 'FD', '?']
118:['+1', '1', '1', '1', '?']
161:['19/13', '1', '19/13', '10/22', '?', 'a.7/24;b.7/25;c.5/26;d.7/26;']
297:['√5', '√55', '11√5', '11√55', '?']
367:['0', '3', '1', '6', '√2', '12', '?', '?', '2', '48']
389:['1/2', '√3/2', '√3/3', '?']
411:['(√5', '1)/4', '1/3', '(√3', '1)/2', '√2', '1', '?']
470:['A', 'C', 'F', 'J', 'O', '?', 'What', 'letter', 'comes', 'next?']
487:['72', '(68)', '41', '28', '(98)', '16', '34', '?', '56']
489:['4342', '(3176)', '1726', '7995', '(7516)', '2162', '8418', '?', '1725']
491:['3', '+6', '+2', '3', '+7', '12', '?']
608:['961', '(25)', '432', '932', '?', '731']
609:['16', '(96)', '12', '10', '?', '7.5']
610:['The', 'following', 'series', 'of', 'numbers', 'contains', 'one', 'number', 'that', 'does', 'not', 'fit', 'the', 'pattern', 'set', 'by', 'the', 'others.', 'What', 'number', 'does', 'not', 'fit?', '3', '5', '7', '11', '14', '17']
615

In [26]:
#get only good sequences
gs = remBadSeq(split_seq)
for k,v in gs.items():
    print(str(k) + ":" + str(v))
print("# Good sequences = " + str(len(gs)) + "/" + str(len(sequences)))

0:['7', '9', '1', '5', '?']
1:['3', '2', '5/3', '3/2', '?']
2:['1', '2', '5', '26', '?']
3:['2', '12', '30', '?']
4:['2', '1', '2/3', '1/2', '?']
5:['4', '2', '2', '3', '6', '?']
6:['1', '7', '8', '57', '?']
7:['4', '12', '8', '10', '?']
8:['1/2', '1', '1', '?', '9/11', '11/13']
9:['95', '88', '71', '61', '50', '?']
10:['2', '6', '13', '39', '15', '45', '23', '?']
11:['1', '3', '3', '5', '7', '9', '13', '15', '?', '?']
12:['1', '2', '8', '28', '?']
13:['0', '4', '18', '?', '100']
14:['1', '1', '2', '2', '3', '4', '3', '5', '?']
15:['1', '52', '313', '174', '?']
16:['5', '15', '10', '215', '?']
17:['7', '0', '1', '2', '9', '?']
18:['0', '1', '3', '10', '?']
19:['9/2', '14', '65/2', '?', '217/2']
20:['1', '1', '2', '6', '24', '?']
21:['3', '4', '8', '24', '88', '?']
22:['20', '22', '25', '30', '37', '?']
23:['1/9', '2/27', '1/27']
25:['1', '2', '4', '8', '16', '?']
26:['2', '1', '2/3', '1/2', '?']
27:['1', '1', '3', '7', '17', '41', '?']
28:['5/2', '5', '25/2', '75/2', '?']
29:['6', '15'

In [27]:
#converts all sequence items to float values
def floatConv(s):
    try:
        return float(s)
    except ValueError:
        try:
            num, denom = s.split('/')
            return float(num) / float(denom)
        except ValueError:
            return 'X'   #bad

#converts all the sequences to floats
def seq2Float(seq):
    floatSeq = {}
    for i in seq.keys():
        s = list(map(lambda x: floatConv(x) if x != "?" else x, seq[i]))
        if "X" in s:  #found a bad one (don't use)
            continue
        else:
            floatSeq[i] = s
    return floatSeq
print(seq2Float(gs))

{0: [7.0, 9.0, 1.0, 5.0, '?'], 1: [3.0, 2.0, 1.6666666666666667, 1.5, '?'], 2: [1.0, 2.0, 5.0, 26.0, '?'], 3: [2.0, 12.0, 30.0, '?'], 4: [2.0, 1.0, 0.6666666666666666, 0.5, '?'], 5: [4.0, 2.0, 2.0, 3.0, 6.0, '?'], 6: [1.0, 7.0, 8.0, 57.0, '?'], 7: [4.0, 12.0, 8.0, 10.0, '?'], 8: [0.5, 1.0, 1.0, '?', 0.8181818181818182, 0.8461538461538461], 9: [95.0, 88.0, 71.0, 61.0, 50.0, '?'], 10: [2.0, 6.0, 13.0, 39.0, 15.0, 45.0, 23.0, '?'], 11: [1.0, 3.0, 3.0, 5.0, 7.0, 9.0, 13.0, 15.0, '?', '?'], 12: [1.0, 2.0, 8.0, 28.0, '?'], 13: [0.0, 4.0, 18.0, '?', 100.0], 14: [1.0, 1.0, 2.0, 2.0, 3.0, 4.0, 3.0, 5.0, '?'], 15: [1.0, 52.0, 313.0, 174.0, '?'], 16: [5.0, 15.0, 10.0, 215.0, '?'], 17: [7.0, 0.0, 1.0, 2.0, 9.0, '?'], 18: [0.0, 1.0, 3.0, 10.0, '?'], 19: [4.5, 14.0, 32.5, '?', 108.5], 20: [1.0, 1.0, 2.0, 6.0, 24.0, '?'], 21: [3.0, 4.0, 8.0, 24.0, 88.0, '?'], 22: [20.0, 22.0, 25.0, 30.0, 37.0, '?'], 23: [0.1111111111111111, 0.07407407407407407, 0.037037037037037035], 25: [1.0, 2.0, 4.0, 8.0, 16.0, '?

## Classifier

In [159]:
valid = seq2Float(gs)  #good sequences dictionary
len(valid)

1033

In [160]:
# data_for_class = []
# for key in valid.keys():
#     data_for_class

In [161]:
maxx = 0
for key in valid.keys():
    if len(valid.get(key))>maxx:
        maxx = len(valid.get(key))
maxx

17

In [162]:
secuencias = []
clases = []
for key in valid.keys():
    if key in [82, 257, 298, 306, 451, 452, 463, 464,468, 476,495, 498,499, 500,504,505,623,624,626,627,628, 
               23, 501, 639, 697, 1056, 1073,
               11, 134, 158, 253, 467, 470, 475, 636, 769, 793, 816, 818, 833, 856, 862, 945]:
        continue


    secuencias.append(valid.get(key))
    clases.append(seqDataAns.classes[key])

In [163]:
question = [] #position of question mark
seq_len = []
for i in range(len(secuencias)):
    question.append(secuencias[i].index('?'))
    seq_len.append(len(secuencias[i]))
    


In [164]:
secuencias2 = secuencias[:] #replace '?' by zero
for i in range(len(secuencias2)):
    secuencias2[i][question[i]] = 0
    
    

In [165]:
secuencias3 = pad_sequences(secuencias2, padding='post') #make all sequences same lenght adding zeros

In [166]:
secuencias3 = secuencias3.tolist()


In [169]:
mask = []
for i in range(len(secuencias3)):
    mask.append([0] * len(secuencias3[0])) 

In [171]:
for i in range(len(mask)):
    for j in range(seq_len[i]):
        mask[i][j] = 1  #assign 1 in the mask sequence for actual values in sequence

In [172]:
for i in range(len(mask)):
    mask[i][question[i]] = 2 #assign 2 in the mask sequence for question mark position

In [173]:
#concatenate the sequence and the mask
secuencias4 = []
for i in range(len(secuencias3)):
    secuencias4.append(secuencias3[i]+mask[i])

In [174]:
len(clases)

993

## Classifier

In [231]:
#change question mark with '-1' to use the single serie
secuencias5 = secuencias3[:] 
for i in range(len(secuencias5)):
    secuencias3[i][question[i]] = -1


In [232]:
X_train, X_test, y_train, y_test = train_test_split(secuencias4, clases, test_size=0.2, random_state=42) #train using the mask
#X_train, X_test, y_train, y_test = train_test_split(secuencias5, clases, test_size=0.2, random_state=42) #train with just the serie


In [233]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [234]:
X_train = X_train.reshape((794, 34, 1))
X_test = X_test.reshape((199, 34,1))

#X_train = X_train.reshape((794, 17, 1))
#X_test = X_test.reshape((199, 17,1))


y_test = pd.get_dummies(y_test)
y_train = pd.get_dummies(y_train).values


In [235]:
y_test.columns

Int64Index([0, 2, 3], dtype='int64')

In [236]:
y_test[1] = 0
y_test
cols =[0,1,2,3]
y_test = y_test[cols]

In [237]:
model = Sequential()
#model.add(LSTM(100, input_shape= (34, 1)))
model.add(LSTM(100, input_shape= (34, 1)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 16)

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 100)               40800     
_________________________________________________________________
dense_47 (Dense)             (None, 32)                3232      
_________________________________________________________________
dense_48 (Dense)             (None, 4)                 132       
Total params: 44,164
Trainable params: 44,164
Non-trainable params: 0
_________________________________________________________________
None
Train on 794 samples, validate on 199 samples
Epoch 1/10
794/794 [==============================] - 1s 2ms/step - loss: 0.4637 - accuracy: 0.7437 - val_loss: 0.4284 - val_accuracy: 0.7638
Epoch 2/10
794/794 [==============================] - 1s 932us/step - loss: 0.4540 - accuracy: 0.7459 - val_loss: 0.4374 - val_accuracy: 0.7500
Epoch 3/10
794/794 [======================

In [238]:
model.save("classifier")
#reconstructed_model = keras.models.load_model("classifier")